In [176]:

import pandas as pd
import nltk
import numpy as np
import string
import enchant
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from ftfy import fix_text
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from googletrans.client import Translator
from nltk.corpus import wordnet as wordnet
import time


In [177]:
dictionary = enchant.Dict("en_US")
translator = Translator()
lemmatizer = WordNetLemmatizer()
spell = SpellChecker()

In [305]:
data = pd.read_csv("London_hotel_reviews.csv", encoding='latin-1')

In [306]:
df1 = data['Review Text'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))
print(df1)

ReadTimeout: The read operation timed out

In [261]:
data_with_english = data.copy()
data_with_english2 = data.copy()

In [236]:
data_with_english.head()

,Property Name,Review Rating,Review Title,Review Text,Location Of The Reviewer,Date Of Review
0,Apex London Wall Hotel,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...,"Casale Monferrato, Italy",10/20/2012
1,Corinthia Hotel London,5,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,"Savannah, Georgia",3/23/2016
2,The Savoy,5,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,London,7/30/2013
3,Rhodes Hotel,4,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,"Maui, Hawaii",6/2/2012
4,The Savoy,5,Perfection,Form the moment we arrived until we left we ex...,"London, United Kingdom",11/24/2017


In [278]:
data_with_english = data_with_english[['Property Name','Review Rating','Review Title','Review Text']]
data_with_english2 = data_with_english2[['Review Rating','Review Title','Review Text', 'Property Name']]
data_with_english2.head()

,Review Rating,Review Title,Review Text,Property Name
0,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...,Apex London Wall Hotel
1,5,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,Corinthia Hotel London
2,5,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,The Savoy
3,4,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,Rhodes Hotel
4,5,Perfection,Form the moment we arrived until we left we ex...,The Savoy


In [238]:
data_with_english.size

109320

def removePunctuation(str):
    str=fix_text(str)
    str=str.translator(str.maketrans('','',string.punctuation))
    if not str:
        str = " "
    return str

def checkIfEnglish(str):
    newstr = removePunctuation(str)
    if newstr != " ":
        if dictionary.check(newstr):
            return 1
    return 0

list_drop=[]
start_time= time.time()
for index, row in data_with_english.iterrows():
    eachRow = row['Review Text'].split()
    eachTopic = row['Review Title'].split()
    try:
        if not (checkIfEnglish(eachTopic[0]) + checkIfEnglish(eachRow[0]) + checkIfEnglish(eachRow[1]) + checkIfEnglish(eachRow[int(len(eachRow)/2)]) + checkIfEnglish(eachRow[-1])) >=4:
            eachList = [row['Property Name'], row['Review Rating'], row['Review Title'], row['Review Text'], row['Location Of The Reviewer'], row['Date Of Review']]
            list_drop.append(eachList)
            data_with_english.drop(index,inplace=True)
    except Exception:
        data_with_english.drop(index, inplace=True)
print("---%.2f seconds ---"%(time.time() - start_time))

print(list_drop)

In [239]:
non_english_df = pd.DataFrame(list_drop, columns = data_with_english.columns)
non_english_df.head()

,Property Name,Review Rating,Review Title,Review Text


In [240]:
print(len(data_with_english))

27330


In [241]:
data_with_english.head()

,Property Name,Review Rating,Review Title,Review Text
0,Apex London Wall Hotel,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...
1,Corinthia Hotel London,5,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...
2,The Savoy,5,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...
3,Rhodes Hotel,4,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...
4,The Savoy,5,Perfection,Form the moment we arrived until we left we ex...


In [242]:
data_with_english_sub = data_with_english

In [243]:
data_with_english_sub.head()

,Property Name,Review Rating,Review Title,Review Text
0,Apex London Wall Hotel,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...
1,Corinthia Hotel London,5,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...
2,The Savoy,5,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...
3,Rhodes Hotel,4,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...
4,The Savoy,5,Perfection,Form the moment we arrived until we left we ex...


In [244]:
data_with_english_sentence_reviews = pd.DataFrame(data_with_english['Review Text'].str.split('.').apply(pd.Series, 1).stack())


In [245]:
data_with_english_sentence_reviews.head(20)

0
0 0   Siamo stati a Londra per un week end ed abbiam...
  1    | Personale molto gentile e disponibile, came...
  2    | Bagno grande con asciugacapelli, vasca da b...
  3    ma soprattutto pulito! | Ottima posizione seb...
  4    | In ogni caso muoversi non è un problema: la...
  5    | Nel periodo in cui siamo stati noi stavano ...
  6                                                    
1 0   I had a pleasure of staying in this hotel for ...
  1                 This hotel was perfect in every way
  2    Communication with the hotel before staying w...
  3                            Checking in was a breeze
  4    You go through the spectacular lobby with mod...
  5    My room, they gave me an upgrade to junior su...
  6    We had a walk-in closet of the size where you...
  7    The decor was very refined, and oh the bathro...
  8    We were 5 minutes walk to Trafalgar Square, b...
  9              Right outside was Embankment tube stop
  10   We would walk to theater area and to numerous...
  11   | We had buffet breakfast or room service eve...
  12              It was pricy, but my rate included it

In [246]:
data_with_english_sentence_reviews.index = data_with_english_sentence_reviews.index.droplevel(-1)

In [247]:
data_with_english_sentence_reviews.name = 'Review Text'

In [248]:
del data_with_english['Review Text']

In [249]:
data_with_english_split_sentence_reviews = data_with_english.join(data_with_english_sentence_reviews)

In [250]:
data_with_english_split_sentence_reviews.size

983188

In [300]:
data_with_english_split_sentence_reviews.head()

,Review Rating,Review Title,Review Text,Property Name
0,Apex London Wall Hotel,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...
0,Apex London Wall Hotel,5,Ottima qualità prezzo,"| Personale molto gentile e disponibile, came..."
0,Apex London Wall Hotel,5,Ottima qualità prezzo,"| Bagno grande con asciugacapelli, vasca da b..."
0,Apex London Wall Hotel,5,Ottima qualità prezzo,ma soprattutto pulito! | Ottima posizione seb...
0,Apex London Wall Hotel,5,Ottima qualità prezzo,| In ogni caso muoversi non è un problema: la...


In [301]:
data_with_english_sentence_reviews = data_with_english_split_sentence_reviews

data_with_english_sentence_reviews.columns = ['Review Rating' , 'Review Title', 'Review Text', 'Property Name']

In [302]:
data_with_english_sentence_reviews.head()

,Review Rating,Review Title,Review Text,Property Name
0,Apex London Wall Hotel,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...
0,Apex London Wall Hotel,5,Ottima qualità prezzo,"| Personale molto gentile e disponibile, came..."
0,Apex London Wall Hotel,5,Ottima qualità prezzo,"| Bagno grande con asciugacapelli, vasca da b..."
0,Apex London Wall Hotel,5,Ottima qualità prezzo,ma soprattutto pulito! | Ottima posizione seb...
0,Apex London Wall Hotel,5,Ottima qualità prezzo,| In ogni caso muoversi non è un problema: la...


In [287]:
def bdp(text):
    tokens = word_tokenize(text)
    tokens=[w.lower() for w in tokens]
    tokens_clean = str.maketrans('','',string.punctuation)
    tokens_cleaned = [w.translate(tokens_clean) for w in tokens]
    clean_words = [word for word in tokens_cleaned if word.isalpha()]
    final_review = " ".join(clean_words)
    return final_review

In [293]:
def adp(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [spell.correction(word) for word in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

In [294]:
import multiprocessing as mp

In [295]:
review_text = []
review_text_sentence = []
review_title = []
review_title_sentence = []

In [296]:
data_with_english2.head()

,Review Rating,Review Title,Review Text,Property Name
0,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...,Apex London Wall Hotel
1,5,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,Corinthia Hotel London
2,5,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,The Savoy
3,4,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,Rhodes Hotel
4,5,Perfection,Form the moment we arrived until we left we ex...,The Savoy


In [304]:
data_with_english_split_sentence_reviews.head()

,Review Rating,Review Title,Review Text,Property Name
0,Apex London Wall Hotel,5,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...
0,Apex London Wall Hotel,5,Ottima qualità prezzo,"| Personale molto gentile e disponibile, came..."
0,Apex London Wall Hotel,5,Ottima qualità prezzo,"| Bagno grande con asciugacapelli, vasca da b..."
0,Apex London Wall Hotel,5,Ottima qualità prezzo,ma soprattutto pulito! | Ottima posizione seb...
0,Apex London Wall Hotel,5,Ottima qualità prezzo,| In ogni caso muoversi non è un problema: la...


In [303]:
start_time = time.time()
with mp.Pool() as pool:
    review_text = pool.map(bdp, data_with_english2["Review Text"])
    review_text = pool.map(adp, review_text)
    review_title = pool.map(bdp, data_with_english2["Review Title"])
    review_title = pool.map(adp, review_title)
    review_text_sentence = pool.map(bdp, data_with_english_split_sentence_reviews['Review Text'])
    review_text_sentence = pool.map(adp, review_text_sentence)
    review_title_sentence = pool.map(bdp, data_with_english_split_sentence_reviews['Review Title'])
    review_title_sentence = pool.map(adp, review_title_sentence)
print("---.2f---" %(time.time() - start_time))
    
    

TypeError: expected string or bytes-like object